# 1. 머신 러닝의 네 가지 분류

- 이전에 다루었던 문제들
    1. 이진 분류 (리뷰 긍정, 부정 분류)
    2. 다중 분류 (기사 토픽 분류)
    3. 스칼라 회귀 (주택값 예측)

- 위의 학습 방법들은 모두 지도 학습(supervised learning)의 예이다.

- 지도 학습
    - 훈련 데이터의 입력과 타깃 사이의 관계를 학습을 목적으로 학습하는 것

## 1.1. 지도 학습

- 가장 흔한 경우
- 타깃에 입력 데이터를 매핑하는 방법을 학습


- 특이한 경우
    1. 시퀸스 생성 : 사진이 주어지면 이를 설명하는 캡션 생성
    2. 구문 트리 예측 : 문장이 주어지면 분해된 구문 트리 예측
    3. 물체 감지 : 사진이 주어지면 사진 안의 특정 물체 주위에 경계 상자를 그린다.
    4. 이미지 분할 : 사진이 주어졌으 때 픽셀 단위로 특정 물체에 마스킹

## 1.2. 비지도 학습

- 어떤 타깃도 사용하지 않고 입력 데이터에 대한 변환을 찾는다.
- 데이터 시각화, 데이터 압축, 데이터 노이즈 제거, 데이터의 상관관계를 알기 위해 사용
- 데이터 분석에 중요한 요소이다.
- 잘 알려진 것
    1. 차원 축소
    2. 군집(clustering)

## 1.3. 자기 지도 학습

- 지도 학습의 특별한 경우
- 사람이 만든 레이블을 사용하지 않는다. -> 사람이 개입하지 않는다.
- 레이블은 필요하지만 보통 경험적인 알고리즘을 사용하여 데이터로부터 생성한다.
- 오토 인코더가 잘 알려진 자기 지도 학습의 예이다.

## 1.4. 강화 학습

- 에이전트는 환경에 대한 정보를 받아 보상을 최대화하는 행동을 선택한다.
- 대부분 연구 영역에 속함

# 2. 머신 러닝 모델 평가

- 훈련에 사용한 데이터로 모델을 평가하지 않는 이유 : 훈련 데이터에 과대적합을 할 수 있기 때문
- 머신 러닝의 목표 : 일반화된 모델을 얻는 것    

## 2.1. 훈련, 검증, 테스트 셋

- 훈련 셋과 테스트 셋 2개를 사용하지 않고 검증 셋을 추가해서 사용하는 이유
    1. 모델을 개발할 때 모델의 설정을 튜닝하기 때문
    2. 검증 셋의 성능을 기반으로 모델의 설정을 튜닝하면 검증 셋에 과대적합 할 수 있기 때문 -> 정보 누설
    
    
- 데이터가 적을 때 훈련, 검증, 테스트 셋으로 나누는 고급 기법
    1. 단순 홀드아웃 검증
    2. K-겹 교차 검증
    3. 셔플링을 사용한 반복 K-겹 교차 검증

### 단순 홀드아웃 검증

- 데이터의 일정량을 테스트 셋으로 떼어 놓는다.
- 정보 누설을 막기 위해 테스트 셋을 사용하여 모델을 튜닝하는것은 안된다.


- 단순 홀드아웃 검증 그림 예시
![홀드_아웃_검증](./images/hold-out.png)


### 홀드아웃 검증 구현 예

In [1]:
"""
num_validation_samples = 10000

np.random.shuffle(data) # 데이터를 무작위로 섞는것이 좋다. -> 오버 피팅 방지를 위해서

validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]

training_data = data[:]

model = get_model()
model.train(training_data) # 훈련 데이터로 훈련 후 
validation_score = model.evaluate(validation_data) # 검증 데이터로 평가

# 반복적으로 훈련 및 튜닝

model = get_model()
model.train(np.concatenate([training_data, validation_data])) # 튜닝이 끝나면 전체 데이터를 가지고 훈련

test_score = model.evaluate(test_data)
"""

'\nnum_validation_samples = 10000\n\nnp.random.shuffle(data) # 데이터를 무작위로 섞는것이 좋다. -> 오버 피팅 방지를 위해서\n\nvalidation_data = data[:num_validation_samples]\ndata = data[num_validation_samples:]\n\ntraining_data = data[:]\n\nmodel = get_model()\nmodel.train(training_data) # 훈련 데이터로 훈련 후 \nvalidation_score = model.evaluate(validation_data) # 검증 데이터로 평가\n\n# 반복적으로 훈련 및 튜닝\n\nmodel = get_model()\nmodel.train(np.concatenate([training_data, validation_data])) # 튜닝이 끝나면 전체 데이터를 가지고 훈련\n\ntest_score = model.evaluate(test_data)\n'

- 단순 홀드아웃의 단점
    1. 데이터가 적을 때는 검증 셋과 테스트 셋이 너무 적어 주어진 전체 데이터를 통계적으로 대표하지 못할 수 있다.

### K-겹 교차 검증

- 데이터가 너무 적어 검증, 테스트 셋이 통계적으로 대표하지 못할 때 사용


- 동일한 크기를 가진 K개 분할로 데이터를 나눈다.
- 최종적으로는 K개의 점수 평균을 이용한다.


- K-겹 교차 검증 사진 예시
![kfold](./images/kfold.jpg)


### K-겹 교차 검증 구현 예

In [2]:
"""
k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []

for fold in range(k):
    validation_data = data[num_validation_samples * fold: num_validation_samples * (fold + 1)] # 검증 셋 선택
    training_data = data[:num_validation_samples * fold] + data[num_validation_samples * (fold + 1)] # np.concatenate로도 만들 수 있다.
    
    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)
    
    validation_scroe = np.average(validation_score)
    
    model = get_model()
    model.train(data)
    test_score = model.evaluate(test_data)
"""

'\nk = 4\nnum_validation_samples = len(data) // k\n\nnp.random.shuffle(data)\n\nvalidation_scores = []\n\nfor fold in range(k):\n    validation_data = data[num_validation_samples * fold: num_validation_samples * (fold + 1)] # 검증 셋 선택\n    training_data = data[:num_validation_samples * fold] + data[num_validation_samples * (fold + 1)] # np.concatenate로도 만들 수 있다.\n    \n    model = get_model()\n    model.train(training_data)\n    validation_score = model.evaluate(validation_data)\n    validation_scores.append(validation_score)\n    \n    validation_scroe = np.average(validation_score)\n    \n    model = get_model()\n    model.train(data)\n    test_score = model.evaluate(test_data)\n'

### 셔플링을 사용한 K-겹 교차 검증

- K 개로 분할을하기 전에 매번 데이터를 무작위로 섞는다.
- 비용이 매우 많이 든다.

## 2.2. 기억해야 할 것

- 평가 방식을 선택할 때
    1. 대표성이 있는 데이터 : 훈련 셋과 테스트 셋은 데이터에 대한 대표성이 있어야한다.
    2. 시간의 방향 : 과거에서 미래를 예측한다면 데이터를 무작위로 섞어서는 안 된다.
    3. 데이터 중복 : 한 데이터셋에 어떤 데이터 포인트가 두 번 등장하면, 데이터를 섞고 훈련 셋과 검증 셋으로 나누었을 때 훈련 셋과 검증 셋에 데이터 포인트가 중복될 수 있다.

# 3. 데이터 전처리, 특성 공학, 특성 학습


## 3.1. 신경망을 위한 데이터 전처리

- 전처리의 목적
    - 원본 데이터를 신경망에 주입하기 쉽게 만드는것
    
- 전처리의 종류
    1. 벡터화
    2. 정규화
    3. 누락된 값 다루기
    4. 특성 추출
   

### 벡터화

- 신경망의 입력과 타깃은 부동 소수 데이터로 이루어진 텐서여야 한다.
- 입력 데이터를 텐서로 변화시키는 것을 데이터 벡터화라 한다.

### 값 정규화

- 그레이스케일 인코딩의 경우 0~255 사이의 정수 값으로 인코딩 되어있다.
- 위와 같은 값들을 255로 나누어서 0~1 상이의 부동 소수 값으로 만드는것을 값 정규화라고 한다.
- 주택 가격 예측시 특성들의 범위가 제각각이어서 입력 값들을 독립적으로 정규화하여 평군이 0 표준편차가 1이되게 해주었다. 이도 값 정규화이다.


- 네트워크를 쉽게 학습시키기 위해서 따라야하는 특징
    1. 작은 값을 취한다.( 0~1 사이의 값을 가져야한다.)
    2. 균일해야 한다. ( 모든 특성이 대체로 비슷한 범위를 가져야 한다.)

### 누락된 값 다루기

- 신경망에서 0에 할당된 값이 없다면 누락된 값을 0으로 설정하여도 좋다.
- 테스트 데이터에 누락된 값이 포함될 가능성이 있다면, 훈련 데이터에도 누락된 값을 가지고 있어야한다.

## 3.2. 특성 공학

- 데이터와 머신 러닝 알고리즘에 관한 지식을 사용하는 단계
- 모델에 데이터를 주입하기 전에 하드코딩된 변환을 적용하여 알고리즘이 더 잘 수행되도록 만들어 준다.

# 4. 과대적합과 과소적합

- 훈련 데이터에 너무 적합하게 된 학습
- 가장 좋은 해결 방법은 더 많은 학습 데이터를 모으는 것
- 정보의 양을 조절, 저장할 수 있는 정보에 제약을 가하는것 -> 규제


- 과소 적합
    - 훈련 데이터의 손실이 낮을수록 테스트 데이터의 손실이 낮은 것

## 4.1. 네트워크 크기 축소

- 과대적합을 막는 가장 단순한 방법 -> 모델의 크기, 학습 파라미터의 수를 줄이는 것
- 손실을 최소화하기 위해 타깃에 대한 예측 성능을 가진 압축된 표현을 학습해야 한다.
- 모델의 기억 용량이 너무 크지도 작지도 않은 부분을 찾아야한다.

### 영화 리뷰 분류 모델 - 모델 크기를 다르게 비교

In [3]:
from keras import models
from keras import layers
from keras.datasets import imdb

import numpy as np

# 데이터 로드
(train_data, train_targets), (test_data, test_targets) = imdb.load_data(num_words=10000)

Using TensorFlow backend.


17465344/17464789 [==============================] - 17s 1us/step


In [4]:
# 입력 데이터 벡터화
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1. # 특정 인덱스의 위치를 1로 만듬
        
    return results

In [5]:
train_data = vectorize_sequences(train_data)
test_data = vectorize_sequences(test_data)

# 타겟을 벡터로 변환
train_targets = np.array(train_targets).astype('float32')
test_targets = np.array(test_targets).astype('float32')

# 검증 데이터 생성
val_train_data = train_data[:10000]
val_train_targets = train_targets[:10000]

partial_train_data = train_data[10000:]
partial_train_targets = train_targets[10000:]

### 원본 모델과, 작은 모델 설정

In [6]:
def original_model():
    model = models.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                 loss = 'binary_crossentropy',
                 metrics=['acc'])
    
    return model

In [7]:
def small_model():
    model = models.Sequential()
    model.add(layers.Dense(6, activation='relu', input_shape=(10000,)))
    model.add(layers.Dense(6, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                 loss = 'binary_crossentropy',
                 metrics=['acc'])
    
    return model

### 각 모델의 훈련

In [8]:
Omodel = original_model()

O_history = Omodel.fit(partial_train_data, partial_train_targets,
                    epochs=20,
                    validation_data=(val_train_data, val_train_targets),
                    batch_size = 512)

Train on 15000 samples, validate on 10000 samples
Epoch 1/20
15000/15000 [==============================] - 3s 171us/step - loss: 0.4941 - acc: 0.7949 - val_loss: 0.3624 - val_acc: 0.8735
Epoch 2/20
15000/15000 [==============================] - 2s 156us/step - loss: 0.2889 - acc: 0.9039 - val_loss: 0.2968 - val_acc: 0.8864
Epoch 3/20
15000/15000 [==============================] - 2s 154us/step - loss: 0.2096 - acc: 0.9319 - val_loss: 0.2774 - val_acc: 0.8903
Epoch 4/20
15000/15000 [==============================] - 2s 154us/step - loss: 0.1632 - acc: 0.9471 - val_loss: 0.2755 - val_acc: 0.8889
Epoch 5/20
15000/15000 [==============================] - 2s 154us/step - loss: 0.1356 - acc: 0.9549 - val_loss: 0.2927 - val_acc: 0.8867
Epoch 6/20
15000/15000 [==============================] - 2s 153us/step - loss: 0.1120 - acc: 0.9657 - val_loss: 0.3012 - val_acc: 0.8835
Epoch 7/20
15000/15000 [==============================] - 2s 154us/step - loss: 0.0895 - acc: 0.9741 - val_loss: 0.3194 - 

In [9]:
Smodel = small_model()

S_history = Smodel.fit(partial_train_data, partial_train_targets,
                    epochs=20,
                    validation_data=(val_train_data, val_train_targets),
                    batch_size = 512)

Train on 15000 samples, validate on 10000 samples
Epoch 1/20
13312/15000 [=========================>....] - ETA: 0s - loss: 0.6404 - acc: 0.6369

KeyboardInterrupt: 

### 검증 손실 비교

In [ ]:
import matplotlib.pyplot as plt

original_val_loss = O_history.history['val_loss']
smaller_val_loss = S_history.history['val_loss']

epochs = range(1, len(original_val_loss) + 1)

plt.plot(epochs, original_val_loss, 'b+', label='original model') # x축에 epochs 값으로, y축을 loss 값으로 설정
plt.plot(epochs, smaller_val_loss, 'bo', label='smaller model')
plt.xlabel('Epochs') # 축 라벨을 Epochs로 설정
plt.ylabel('Validation loss')
plt.legend()

plt.show()

- 기존 네트워크보다 작은 모델이 더 느게 과대 적합 된다.


### 더 큰 모델 설정

In [ ]:
def bigger_model():
    model = models.Sequential()
    model.add(layers.Dense(1024, activation='relu', input_shape=(10000,)))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                 loss = 'binary_crossentropy',
                 metrics=['acc'])
    
    return model

In [ ]:
Bmodel = bigger_model()

B_history = Bmodel.fit(partial_train_data, partial_train_targets,
                    epochs=20,
                    validation_data=(val_train_data, val_train_targets),
                    batch_size = 512)

In [ ]:
plt.clf()

original_val_loss = O_history.history['val_loss']
bigger_val_loss = B_history.history['val_loss']

epochs = range(1, len(original_val_loss) + 1)

plt.plot(epochs, original_val_loss, 'b+', label='original model') # x축에 epochs 값으로, y축을 loss 값으로 설정
plt.plot(epochs, bigger_val_loss, 'bo', label='bigger model')
plt.xlabel('Epochs') # 축 라벨을 Epochs로 설정
plt.ylabel('Validation loss')
plt.legend()

plt.show()

- 큰 모델이 더욱 빠르게 과대적합하는것을 볼 수 있다.

## 4.2. 가중치 규제 추가

- 오캄의 면도날
    - 어떤 것에 대한 두 가지의 설명이 있다면 더  적은 가정이 필요한 간단한 설명이 옳다.
    
- 간단한 모델이 복잡한 모델보다 덜 과대적합될 가능성이 높다.


- 가중치 규제
    - 네트워크 복잡도에 제한을 두어 가중치가 작은 값을 가지도록 강제하는 것
    
    
- 가중치 규제의 종류
    1. L1 규제 : 가중치의 절대값에 비례하는 비용이 추가
    2. L2 규제 : 가중치의 제곱에 비례하는 비용이 추가 ( 가중치 감쇠라고도 불림)
    

### 모델에 L2 가중치 추가하기

In [ ]:
from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                      activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

- l2(0.001)은 가중치 행렬의 모든 원소를 제곱하고 0.001을 곱하여 네트워크의 전체 손실에 더해진다는 의미이다.

In [ ]:
model.compile(optimizer='rmsprop',
                 loss = 'binary_crossentropy',
                 metrics=['acc'])

L2_history = model.fit(partial_train_data, partial_train_targets,
                    epochs=20,
                    validation_data=(val_train_data, val_train_targets),
                    batch_size = 512)

In [ ]:
plt.clf()

original_val_loss = O_history.history['val_loss']
L2_val_loss = L2_history.history['val_loss']

epochs = range(1, len(original_val_loss) + 1)

plt.plot(epochs, original_val_loss, 'b+', label='original model') # x축에 epochs 값으로, y축을 loss 값으로 설정
plt.plot(epochs, L2_val_loss, 'bo', label='L2-regularized model')
plt.xlabel('Epochs') # 축 라벨을 Epochs로 설정
plt.ylabel('Validation loss')
plt.legend()

plt.show()

- 원본 모델보다 가중치 규제를 사용한 모델이 과대적합에 잘 견디는것을 볼 수 있다.
- L2 규제 말고도 L1 규제, L1_L2 규제 병행을 사용 할 수도 있다.

## 4.3. 드롭아웃 추가

- 신경망을 위해 사용되는 규제 기법 중에서 가장 효과적이고 널리 사용되는 방법 중 하나
- 훈련하는 도안 무작위로 층의 일부 출력 특성을 제외시킨다.
- 뉴런들의 부정한 협업을 방지하고 과대적합을 방해한다.
- 중요하지 않은 우연한 패턴을 깨트린다.


### IMDB 네트워크에 드롭아웃 추가하기

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5)) # 50%를 무시하겠다는 의미
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
                 loss = 'binary_crossentropy',
                 metrics=['acc'])

Droout_history = model.fit(partial_train_data, partial_train_targets,
                    epochs=20,
                    validation_data=(val_train_data, val_train_targets),
                    batch_size = 512)

In [ ]:
plt.clf()

original_val_loss = O_history.history['val_loss']
D_val_loss = Droout_history.history['val_loss']

epochs = range(1, len(original_val_loss) + 1)

plt.plot(epochs, original_val_loss, 'b+', label='original model') # x축에 epochs 값으로, y축을 loss 값으로 설정
plt.plot(epochs, D_val_loss, 'bo', label='Dropout model')
plt.xlabel('Epochs') # 축 라벨을 Epochs로 설정
plt.ylabel('Validation loss')
plt.legend()

plt.show()

- 기본 네트워크보다 성능이 더 향상된 것을 볼 수 있다.


### 과대적합을 방지하기 위해서 사용되는 방법들
1. 훈련 데이터를 더 모은다.
2. 네트워크 용량을 감소시킨다.
3. 가중치 규제를 추가한다.
4. 드롭아웃을 추가한다.

# 5. 보편적인 머신 러닝 작업 흐름


## 5.1. 문제 정의와 데이터셋 수집

- 주어진 문제 정의
    1. 가용데이터의 유무
    2. 문제의 유형(이진 분류, 다중 분류, 회귀, 군집....)
    
    
- 입력과 출력
    1. 주어진 입력으로 출력을 예측할 수 있다고 가설을 세움
    2. 가용한 데이터에 입력과 출력 사이의 관계를 학습하는 데 충분한 정보가 있다고 가설을 세운다.

## 5.2. 성공 지표 선택

- 클래스 분포가 균일한 분류 문제에서는 정확도, ROC AUC가 일반적인 지표
- 랭킹 문제나 다중 레이블 문제에서는 평균 정밀도를 사용할 수 있다.

## 5.3. 평가 방법 선택

- 앞서 잘 알려진 세 가지의 평가 방식이 있다.
    1. 홀드아웃 검증 셋 분리
    2. K-겹 교차 검증
    3. 반복 K-겹 교차 검증
 
- 대부분의 경우 첫 번째로 충분하다.

## 5.4. 데이터 준비

- 머신 러닝 모델을 심층 신경망이라고 가정
- 데이터는 텐서로 구성
- 텐서는 일반적으로 작은 값 (0~1) 혹은 (-1,1) 범위로 되어있다.
- 특성마다 범위가 다르면 정규화를 한다.
- 특성 공학을 수행 할 수 있다.

## 5.5. 기본보다 나은 모델 훈련하기

- 이 단계의 목표
    - 통계적 검정력을 달성하는 것(아주 단순한 모델보다 나은 수준의 작은 모델을 개발)


- 통계적 검정력을 달성하는 것이 항상 가능하지는 않다.
    - 여러개의 타당성 있는 네트워크 구조를 시도해 보고 무작위로 예측하는 모델보다 낫지 않다면 문제 정의에서 한 2개의 가설이 틀렸을 수 있다.
    - 가설이 틀렸을 경우 기획부터 다시 해야 한다.
    
    
- 일이 잘 진행되었다면 모델을 만들기 위해 세 가지 중요한 선택을 한다.
    1. 마지막 층의 활성화 함수 : 네트워크 출력에 필요한 제한을 가한다.
    2. 손실 함수 : 풀려고 하는 문제의 종류에 적합해야 한다.(이진 분류:binary_crossentropy, 회귀: mse)
    3. 최적화 설정: 어떤 옵티마이저를 사용하는가?, 학습률은 어떻게 설정하였는가를 생각한다.

|문제 유형|마지막 층의 활성화 함수|손실 함수|
|:---:|:---:|:---:|
|이진 분류|시그모이드|binary_crossentropy|
|단일 레이블 다중 분류|소프트맥스|categorical_crossentropy|
|다중 레이블 다중 분류|시그모이드|binary_crossentropy|
|임의 값에 대한 회귀|없음|mse|
|0과 1 사이 값에 대한 회귀|시그모이드|mse 또는 binary_crossentropy|

## 5.6. 몸집 키우기: 과대적합 모델 구축

- 통계적 검정력을 가진 모델을 얻음 -> 충분히 성능을 내는지 확인
- 또한 적절한 용량을 찾기 위해
    1. 층을 추가
    2. 층의 크기를 키움
    3. 더 많은 에포크로 훈련
- 위 과정을 이용해 과대적합한 모델을 만들고 용량을 줄여나간다.

## 5.7. 모델 규제와 하이퍼파라미터 튜닝

- 이 단계가 대부분의 시간을 차지
- 모델을 수정할 때 적용해 볼 것들
    1. 드롭아웃 추가
    2. 층을 추가하거나 제거, 다른 구조 시도
    3. L1, L2 또는 두가지 모두 추가
    4. 최적의 설정을 찾기 위해 하이퍼파리미터를 바꿈
    5. 선택적으로 특성 공학을 시도